In [1]:
cd ..

/Users/hidde/Documents/GitHub/ma-communicative-robots/projects/memory/notebooks


In [2]:
cd ..

/Users/hidde/Documents/GitHub/ma-communicative-robots/projects/memory


In [3]:
import json
from glob import glob
from tqdm.notebook import tqdm
import os
import random

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("tscholak/t5.1.1.lm100k.base")
model = T5ForConditionalGeneration.from_pretrained("tscholak/t5.1.1.lm100k.base")


def read_json(path):
    """Read json file.

    Args
    ----
    path: path to the json

    Returns
    -------
    loaded: loaded dict

    """
    with open(path, "r") as stream:
        loaded = json.load(stream)

    return loaded

In [4]:
def convert_to_days(entry):
    entry = sorted(entry, key=lambda x: x[-1])
    for idx, mem in enumerate(entry):
        max_len = len(entry)
        days = len(entry) - idx - 1
        if days == 0:
            timestamp = "today"
        else:
            timestamp = f"{days} days ago"
        entry[idx][-1] = timestamp

In [5]:
path = "data/128_64.json"
data = read_json(path)
prompts = []
answers = []
prompt_counter = 0
for entry in data["val"]:
    prompt_counter += 1
    print("prompt: ", prompt_counter)
    entry["episodic_memory_system"] = sorted(
        entry["episodic_memory_system"], key=lambda x: x[-1], reverse=True
    )
    convert_to_days(entry["episodic_memory_system"])

    prompt = []
    counter = 0
    for mem in entry["episodic_memory_system"]:
        if counter == 32:
            break
        prompt.append(f"{mem[0]} was at {mem[2]}, {mem[3]}.")
        counter += 1
    prompt.reverse()
    print("episodic counter", counter)
    counter = 0
    for mem in entry["semantic_memory_system"]:
        prompt.append(f"{mem[0]} were found at {mem[2]}.")
        counter += 1
    print("sementic counter", counter)
    prompt.append(f"Where is {entry['question'][0]}?")
    prompt = " ".join(prompt)
    prompts.append(prompt)
    answers.append(entry["correct_answer"])

prompt:  1
["Mary's bench was at Mary's city, today."]
episodic counter 1
sementic counter 0
prompt:  2
["Mary's bench was at Mary's city, 1 days ago.", "John's bench was at John's building, today."]
episodic counter 2
sementic counter 0
prompt:  3
["Mary's bench was at Mary's city, 2 days ago.", "John's bench was at John's building, 1 days ago.", "Linda's boat was at Linda's water, today."]
episodic counter 3
sementic counter 0
prompt:  4
["Mary's bench was at Mary's city, 3 days ago.", "John's bench was at John's building, 2 days ago.", "Linda's boat was at Linda's water, 1 days ago.", "Mary's person was at Mary's building, today."]
episodic counter 4
sementic counter 0
prompt:  5
["Mary's bench was at Mary's city, 4 days ago.", "John's bench was at John's building, 3 days ago.", "Linda's boat was at Linda's water, 2 days ago.", "Mary's person was at Mary's building, 1 days ago.", "James's car was at James's city, today."]
episodic counter 5
sementic counter 0
prompt:  6
["Mary's ben

In [6]:
def predict(prompts):
    predictions = []
    for prompt in prompts:
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids
        outputs = model.generate(input_ids)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(prediction)
        predictions.append(prediction)
    return predictions

In [7]:
def calculate_accuracy(answers, predictions):
    correct = 0
    wrong = 0
    for answer, pred in zip(answers, predictions):
        if answer in pred:
            correct += 1
        else:
            wrong += 1

    print(f"correct: {correct}\nwrong: {wrong}")
    print(f"accuracy is {correct / (correct+ wrong)*100}")

In [8]:
predictions = predict(prompts)

Mary's bench was at Mary's city, today. Where is Mary's bench
Mary's bench was at Mary's city, 1 days ago. John's bench
The bench was at John's building, 1 days ago.
Mary's person was at Mary's building, today. Where is Mary's person
John's bench was at John's building, 3 days ago. Mary's person


KeyboardInterrupt: 

In [ ]:
calculate_accuracy(answers, predictions)